In [ ]:
from dataclasses import dataclass
from datetime import datetime

@dataclass
class Regexes:
    timestamp = r"(?<=\[)\d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2}(?=\])"
    author = r"(?<=\]\s)(.*?)(?=:)"
    clear = r"\[\d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2}\]\s[~a-zA-Z\s]+:"

In [ ]:
regexes = Regexes()

In [ ]:
from pathlib import Path
datafile = Path("../data/raw/chat.txt")
datafile.exists()

In [ ]:
import re
records = []
with datafile.open() as f:
    for line in f.readlines():
        ts = re.search(regexes.timestamp, line)
        if ts:
            timestamp = datetime.strptime(ts.group(0), "%d-%m-%Y %H:%M:%S")
            msg = re.sub(regexes.clear, "", line)
            author = re.search(regexes.author, line)
            if author:
                author = author.group(0)
            records.append((timestamp, author, msg))
        else:
            msg = msg + re.sub(regexes.clear, "", line)
            records[-1] = (timestamp, author, msg)


In [ ]:
import pandas as pd
df = pd.DataFrame(records, columns=["timestamp", "author", "message"])

In [ ]:
df.to_csv("../data/processed/chat.csv", index=False)

In [ ]:
df.dtypes